In [72]:
from bs4 import BeautifulSoup
import glob
import itertools
import os
from tqdm import tqdm
links = []

for fname in tqdm(glob.glob(os.path.expanduser("~/data/texas_air/tceq/tceq_all_*.html"))):
    with open(fname) as fp:
        soup = BeautifulSoup(fp)
        rows = [d.parent.parent for d in soup.find_all("div", class_="xuiDisplayText_Sm") if "MAERT" in d.text]
        links.extend(["https://records.tceq.texas.gov" + l["href"] 
                      for l in itertools.chain(*[row.find_all("a") for row in rows]) 
                      if "TCEQ_EXTERNAL_SEARCH_GET_FILE" in l["href"]])


100%|██████████| 117/117 [00:10<00:00, 10.97it/s]


In [76]:
import re
len(links), links[0]

(323,
 'https://records.tceq.texas.gov/cs/idcplg?IdcService=TCEQ_EXTERNAL_SEARCH_GET_FILE&dID=5233344&Rendition=Web')

In [82]:
with open("fetch_pdfs.sh", "w") as fp:
    for i, link in enumerate(links):
        id_match = re.search(r"dID=(\d+)", link)
        if id_match:    
            id = id_match.group(1)
        else:
            id = f"UNK_{i}"
        fname = os.path.expanduser(f"~/data/texas_air/tceq/pdf/{id}.pdf")
        fp.write(f"wget {link} -O {fname}\nsleep 2\n")